In [1]:
from omg.datamodule import DataModule
from omg.datamodule.dataloader import OMGTorchDataset
from torch_geometric.loader import DataLoader

In [2]:
import torch

In [3]:
dataset = DataModule.from_lmdb(["./../data/mp_20/train.lmdb"])

example_data = OMGTorchDataset(dataset)

loader = DataLoader(example_data, batch_size=10)

2024-09-25 19:11:11.924 | INFO     | omg.datamodule.datamodule:add_from_lmdb:803 - Reusing existing LMDB file: dataset_274518dcfedeeb30399261097eb4fcb7.lmdb


In [4]:
for batch in loader:
    t = torch.rand(1)
    print(t,batch)
    break   

tensor([0.7945]) OMGDataBatch(n_atoms=[10], batch=[90], species=[90], cell=[30, 3], pos=[90, 3], ptr=[11])


In [5]:
t = t.repeat(10,1)
t

tensor([[0.7945],
        [0.7945],
        [0.7945],
        [0.7945],
        [0.7945],
        [0.7945],
        [0.7945],
        [0.7945],
        [0.7945],
        [0.7945]])

In [6]:
t.dtype

torch.float32

In [7]:
batch.cell = batch.cell.reshape(10,3,3)


In [8]:
batch.cell.shape

torch.Size([10, 3, 3])

In [9]:
#batch.pos = torch.matmul(batch.pos, torch.inverse(batch.cell))

In [10]:
from omg.model.encoders.cspnet_full import CSPNet_Full

/home/pawanprakash/miniconda3/envs/fermat/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
model = CSPNet_Full(latent_dim=1)
model = model.double()

In [12]:
md = model(batch,t = t)

torch.Size([10, 3, 3])
torch.Size([10, 3, 3])
torch.Size([10, 3, 3])
here


In [15]:
md


Data(species_b=[90, 100], species_eta=[90, 100], pos_b=[90, 3], pos_eta=[90, 3], cell_b=[10, 3, 3], cell_eta=[10, 3, 3])

In [22]:
batch.species

tensor([11, 11, 11, 25, 27, 28,  8,  8,  8,  8,  8,  8, 60, 13, 13, 13, 13, 13,
        13, 13, 13, 29, 29, 29, 29, 48, 49, 49, 49, 52, 52, 52, 52, 33, 44, 45,
        45, 45, 45, 23, 23, 23, 23,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,
         9, 70, 50, 56, 56, 48, 55, 37, 37, 69, 17, 17, 17, 17, 17, 17, 67, 14,
        28, 28, 28, 28, 19, 19, 37, 69, 23, 23,  8,  8,  8,  8,  8,  8,  8,  8])

In [17]:
md.species_b

tensor([[-0.8398,  0.7658, -0.7543,  ...,  0.4989,  0.2560,  0.4430],
        [-0.8380,  0.7662, -0.7498,  ...,  0.4955,  0.2609,  0.4403],
        [-0.8330,  0.7642, -0.7513,  ...,  0.4981,  0.2632,  0.4423],
        ...,
        [-0.5885,  0.2447, -0.9283,  ..., -0.9191, -0.6786,  1.5135],
        [-0.5950,  0.2425, -0.9271,  ..., -0.9158, -0.6754,  1.5115],
        [-0.5896,  0.2458, -0.9260,  ..., -0.9212, -0.6766,  1.5143]],
       dtype=torch.float64, grad_fn=<AddmmBackward0>)